# Using MIP to solve the problem

We can also use the even more general _mixed integer programming_ approach to solve the problem. However, solving the problem may take more computation.

## The MIP approach

We assign integer variables to the edges in the graph. The value of each variable in the solution is the flow along the corresponding edge. For an edge between a given worker and a task, the worker is assigned to the task if the value of the variable is $1$ and $0$ otherwise.

Intuitively, this seems the best match for the marketing optimisation problem.



In [2]:
from ortools.linear_solver import pywraplp
import time


## Define the problem

There is one variable for each pair of worker and task. The workers are numbered $0-5$ and the tasks $0-3$.


In [3]:
def main():
    """
    Solving an assignment problem with MIP
    """
    
    # Instantiate a mixed integer solver
    solver = pywraplp.Solver('SolveAssignmentProblemMIP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # define the costs
    cost = [[90, 76, 75, 70],
          [35, 85, 55, 65],
          [125, 95, 90, 105],
          [45, 110, 95, 115],
          [60, 105, 80, 75],
          [45, 65, 110, 95]]
    
    team1 = [0, 2, 4]
    team2 = [1, 3, 5]
    team_max = 2
    
    num_workers = len(cost)
    num_tasks = len(cost[0])
    
    # create the binary integer variables for the problem
    x = {}
    
    for i in range(num_workers):
        for j in range (num_tasks):
            x[i, j] = solver.BoolVar('x[%i, %i]' % (i, j))

    # create the objective function
    solver.Minimize(solver.Sum([cost[i][j] * x[i, j] for i in range(num_workers)
                                                     for j in range(num_tasks)]))
    
    # create the constraints
    ## each worker is assigned to at most 1 task
    for i in range(num_workers):
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)
        
    ## Each task is assigned to exactly 1 worker
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
        
    ## Each team takes on 2 tasks
    solver.Add(solver.Sum([x[i, j] for i in team1 for j in range(num_tasks)]) <= team_max)
    solver.Add(solver.Sum([x[i, j] for i in team2 for j in range(num_tasks)]) <= team_max)
    
    # Invoke the solver
    sol = solver.Solve()
    
    print('Total cost = %d' % (solver.Objective().Value()))
    print()
    for i in range(num_workers):
        for j in range(num_tasks):
            if x[i, j].solution_value() > 0:
                print('worker %d assigned to task %d; cost %d.' % (i, j, cost[i][j]))
    
    print()
    print('Time = ', solver.WallTime(), " milliseconds.")
    
    

In [4]:
if __name__ == "__main__":
    start_time = time.clock()
    main()
    print()
    print("Time = ", time.clock() - start_time)

Total cost = 250

worker 0 assigned to task 2; cost 75.
worker 1 assigned to task 0; cost 35.
worker 4 assigned to task 3; cost 75.
worker 5 assigned to task 1; cost 65.

Time =  45  milliseconds.

Time =  0.0214679999999996
